In [1]:
# merlin_image_browsing.ipynd demonstrates pixel-level operations on LSST lab test images
# using some DMStack functionality
# Merlin Fisher-Levine, 2016

# best to start command line with ipython --pylab if not running inside notebook
# initialize by importing useful stuff
import numpy as np
import matplotlib.pyplot as plt
import astropy as ast
import astropy.stats
from astropy.io import fits
import scipy as sc
import scipy.signal
import textwrap

# Load parts of the LSST stack we will need:
import lsst.afw.image as afwImage # needed for declaring/opening images
import lsst.afw.display.ds9 as ds9 # needed for display
import lsst.obs.decam as obs_decam # needed for DECam
import lsst.daf.persistence as dafPersist # needed for the butler
import lsst.afw.geom as afwGeom # needed for the camera model
import lsst.afw.cameraGeom as cameraGeom # needed for the camera model
# import lsst.afw.geom.ellipses as afwEllipse


# this next magic command makes plots appear within the notebook
%matplotlib inline

# define figure size parameters to make figures larger than default
figwidth=10
figheight=10

print 'Loaded'

Loaded


In [2]:
# This is still the easiest way of examining image metadata (in my opionion, and as things stand)

# # open a FITS file 
# hdulist=fits.open("foo.fits")

# # look at objects that reside within this "list". Note this list is zero-based.
# hdulist.info()


In [3]:
filename = '/nfs/lsst2/photocalData/data/observer2/DECam_00496473.fits.fz'

#Load the file itself
exposure = afwImage.ExposureF(filename)

#This is one of the underlying objects
maskedImage = exposure.getMaskedImage()

# These three are held in the maskedImage
image = maskedImage.getImage()
mask = maskedImage.getMask()
variance = maskedImage.getVariance()

#send that image to ds9 for display
ds9.mtv(image, frame=9)

NameError: name 'HDU_num' is not defined

In [ ]:
# # convert header to string and print header information for primary header 
# print "----------extension 0 header-------------" # common to all HDU elements for this image
# print " "
# print textwrap.fill(str(hdulist[0].header),80) # this slightly obscure syntax makes printout 80 columns wide


In [ ]:
# # print first extension's header
# print "----------extension 1 header-------------" # amplifier-specific header info, mostly pixel placement information
# print "  "
# print textwrap.fill(str(hdulist[1].header),80) 

In [ ]:
# # HDU header information for FITS binary table with photodiode time series
# print "----------extension 20 header-------------" # This pertains to monochromator and monitor diode
# print "  "
# print textwrap.fill(str(hdulist[20].header),80) 

In [ ]:
# # assign the pixel array for extenstion 1 within the FITS file to an array we can use
# pixeldata=hdulist[1].data

# # peek at range of pixel values we have
# print "Maximum pixel value= %s" %np.max(pixeldata)
# print "Minimum pixel value= %s" %np.min(pixeldata)
# print "Median pixel value= %s" %np.median(pixeldata)

In [4]:
# We still have access to the raw pixel information, this is owned by the image, in its array:
print "image format is "
image.getArray().shape
# pixeldata.shape

image format is 


(4146, 2160)

In [ ]:
# can still do all the stuff from last week:
pixeldata = image.getArray()

# peek at some individual pixel values
print "first pixel is            %s" %pixeldata[0,0] # very first pixel digitized, this is actually pre-scan in the image
print "first imaging pixel is   %s" %pixeldata[1,10] # column 11 (index value 10) is first pixel in imaging section
print "pixel near center is     %s" %pixeldata[2000,200] # a pixel near the center of the amplifier's imaging section
print "last pixel in array is    %s" %pixeldata[2047,543] # last pixel in the image, in overscan region
print " "
print "here are some sequential pixels:" # note Python doesn't give the final element from a slice range
subset=pixeldata[2000,201:205]
print(subset)

In [ ]:
# display image as grayscale with colorbar
fig1=plt.figure(1,[figwidth,figheight])
plt.imshow(pixeldata,cmap = 'gray')
plt.colorbar()

In [ ]:
# that's fine, but can we please flip it so first pixel read out is in lower left corner? 
fig1=plt.figure(1,[figwidth,figheight])
plt.imshow(pixeldata,cmap = 'gray',origin='lower')
plt.colorbar()

# OK, that's better but need to remember that index order is still (row,column) and not (x,y)

In [ ]:
# make histogram of pixel values. pixeldata.flat makes 1-d array. Do 3 sigma clipping to exclude outliers

sigma_clipped_pixels = astropy.stats.sigma_clip(pixeldata[0:2002,11:512].flat)
fig2 = plt.figure(2,[figwidth,figheight])
pixel_range = sigma_clipped_pixels.min(), sigma_clipped_pixels.max()
# plt.hist(pixeldata.flat,bins=50,range=pixel_range)
plt.hist(pixeldata.flatten(),bins=50,range=pixel_range) #need a slight tweak here
fig2.suptitle('histogram of 3-sigma clipped pixel values')

# Pixel Response NonUniformity spec is 5% rms. This is same as sigma of the distibution
pixel_sigma=np.std(pixeldata[0:2002,11:512].flat)# use only the imaging area
PRNU=100*(pixel_sigma/np.mean(pixeldata[0:2002,11:512].flat))
print "PRNU for this image is %.1f percent, LSST spec is 5 percent" %PRNU


In [ ]:
# row plot, horizontal cut across columns
fig3=plt.figure(3,[figwidth,figheight])
plt.plot(pixeldata[500,0:543],'b')
plt.xlabel('column number')


# column plot, vertical cut across rows
fig4=plt.figure(4,[figwidth,figheight])
plt.plot(pixeldata[0:2047,200],'r')
plt.xlabel('row number')


# column plot of overscan region, vertical cut across rows
fig5=plt.figure(5,[figwidth,figheight])
plt.plot(pixeldata[0:2047,530],'r')
plt.xlabel('overscan row number')


In [5]:
datapath = '/nfs/lsst2/photocalData/data/decam/rerun/3/'
# datapath = '/nfs/lsst2/photocalData/data/scratch/'
butler = dafPersist.Butler(datapath)

In [8]:
import lsst.afw.cameraGeom.utils as camGeomUtils
camGeomUtils = reload(camGeomUtils)

camera = butler.get("camera")

In [15]:
frame, visit = 1, 496473

product = 'raw'
# other product options include 'raw', calexp', 'postISRCCD'

# these should need to be specified, but a bug is necessitating them at the moment in obs_decam
date = '2015-11-25'
filter = 'r'

imageSource = camGeomUtils.ButlerImage(butler, product, visit=visit, verbose=True, date=date, filter=filter)

# just a way of creating a dictionary to keep these in without overwriting each time we run the cell
try:
    mos
except NameError:
    mos = dict()

mos['496473-raw'] = camGeomUtils.showCamera(camera, imageSource=imageSource, title=str(visit), binSize=8)

Reading 61: No unique lookup for ['hdu'] from {'filter': 'r', 'date': '2015-11-25', 'ccd': 61, 'ccdnum': 61, 'visit': 496473}: 0 matches


In [16]:
ds9.mtv(mos['496473-raw'], frame=2)

In [ ]:
# edge rolloff plot. Flux is diminished at low row numbers, near the serial register. 
fig5=plt.figure(5,[figwidth,figheight])
plt.plot(pixeldata[0:30,200],'ro-')
plt.xlabel("rows, from bottom of array")


In [ ]:
# get better statistics by averaging multiple columns
# picking axis=0 gives average of multiple rows, axis=1 is average of multiple columns
# this averages columns 200 to 299
avgcol=np.mean(pixeldata[0:2047,200:300],axis=1)

# plot edge rolloff using averaged column, better statistics
fig6=plt.figure(6,[figwidth,figheight])
plt.plot(avgcol[0:30],'ro-')
plt.grid()
fig6.suptitle('rows at bottom of image, near serial register')

# what about the upper edge, near the midline of the CCD? 
# this perturbation is due to midline bleed stop implant. It repels charge so top row is deficient but next 
# one catches the charge
fig8=plt.figure(8,[figwidth,figheight])
plt.plot(avgcol[1980:2002],'ro-')
plt.grid()
fig8.suptitle('rows at top of image, midline')

In [ ]:
# what is Response Non-Uniformity in column direction? Only use imaging pixels, presumes uniform illumination
PRNUcol=avgcol[0:2002]/np.mean(avgcol[0:2002])
fig8=plt.figure(8,[figwidth,figheight])
plt.plot(PRNUcol,'ro-')
plt.grid()
fig8.suptitle('PRNU')
plt.xlabel('row ')

# dominated by edge rolloff at bottom and midline bleed artifact at the top. Omit those pixels 
PRNUtrim=PRNUcol[20:1900]
fig9=plt.figure(9,[figwidth,figheight])
plt.plot(PRNUtrim,'ro-')
plt.grid()
fig9.suptitle('trimmed PRNU')
plt.xlabel('row ')

# there is definitely a periodic low frequency modulation, that dominates the response. How about high frequencies? 

In [ ]:
# implement high pass filter by taking median and subtracting it from the column average 
avgcol2=np.mean(pixeldata[15:1950,100:500],axis=1) # create an average column that excludes top and bottom of amplifier
avgcol2_med = sc.signal.medfilt(avgcol2, 51)  # make low-pass filtered version of this using median over 51 pixel box
avgcol2_HF=avgcol2-avgcol2_med # subtract off the low order structure to emphasize small scale structure

# always make sure average is zero before taking an FFT!
avgcol2_HF=avgcol2_HF-np.median(avgcol2_HF) 

# try a spatial Fourier transform 
from scipy.fftpack import fft, fftshift
A = fft(avgcol2_HF, 2048) / (len(avgcol2_HF)/2.0)
freq = np.linspace(-0.5, 0.5, len(A))  # this is cycles per sample, so cycles per pixel. 
FFTavgcol2 = 20 * np.log10(np.abs(fftshift(A / abs(A).max()))) # take log of power, dB units
fig10=plt.figure(10,[figwidth,figheight])
plt.plot(freq, FFTavgcol2)
plt.axis([0, 0.5, -35 ,5])
plt.xlabel("frequency, cycles per pixel") # final element on right is Nyquist-sampled two cycles per pixel
plt.ylabel("log of FFT strength, dB")
plt.grid()
fig10.suptitle('FFT along a column')

# looks like a peak and harmonics at ~10 pixel period, but not exactly locked to pixels. 
# Can we see that in spatial plot? Sort-of...

fig11=plt.figure(11,[figwidth,figheight])
plt.plot(avgcol2_HF[200:300],'ro-')
plt.grid()
fig11.suptitle('expansion of column')
plt.xlabel('row ')

# OK, 

In [ ]:
# what is the temporal stability of the overscan, as each row is read out? 
# make an average column in the overscan region
overscancolavg=np.mean(pixeldata[0:2047,530:543],axis=1)
fig9=plt.figure(9)
plt.plot(overscancolavg,'ro-')
plt.grid()
plt.xlabel('row')

# Just for fun, try a polynomial fit to the overscan. Need a counter for this
rowcountermax=overscancolavg.size
rowcounter=[i for i in range(rowcountermax)]

# fit to a quadratic
quadfit=np.polyfit(rowcounter,overscancolavg,2)

# evaluate the fit
biasfit=np.polyval(quadfit,rowcounter)

# plot fit and data
fig10=plt.figure(10)
plt.plot(rowcounter,overscancolavg,'ro')
plt.plot(rowcounter,biasfit,'b',linewidth=4)
plt.grid()
plt.xlabel('row')

print"signal span of bias region is %s" %(np.max(biasfit)-np.min(biasfit))


In [ ]:
# what about in the other direction? 
row_100_200avg=np.mean(pixeldata[100:200,:],axis=0) # make average of rows 100 to 199

fig11=plt.figure(11,[figwidth,figheight])
plt.plot(row_100_200avg,'bo-')
plt.xlim(xmin=450)
plt.grid()
plt.xlabel('column number')
plt.ylabel('ADU')


In [ ]:
# yuk! is that edge rolloff at the right edge of the imaging array? Look at an interior amp instead...

# pick extension 5 of the FITS file
pixeldata5=hdulist[5].data

# average a few rows together and plot the result
row_100_200avg_5=np.mean(pixeldata5[100:200,:],axis=0)
print "first few values of overscan region are %s" %row_100_200avg_5[520:523]

fig12=plt.figure(12,[figwidth,figheight])
plt.plot(row_100_200avg_5,'bo-')
plt.xlim(xmin=480)


In [ ]:
# looks like 523 to 544 are useful. 
readnoise5=np.std(pixeldata5[:,523:544])
print "read noise from overscan is %.2f ADU" %readnoise5 

In [ ]:
# what happens if we start including pixels that aren't clean bias ones?
print "including first overscan column gives read noise of %.2f ADU" %np.std(pixeldata5[:,522:544])
print "including last data region column gives read noise of %.2f ADU, and that ain't right! " %np.std(pixeldata5[:,521:544]) 


In [ ]:
# try for CTE determination
row_CTE_trimmed=np.mean(pixeldata5[100:200,521:544],axis=0)
row_CTE_trimmed.size

row_CTE_trimmed_debiased=row_CTE_trimmed-np.mean(row_CTE_trimmed[5:23])

# make all numbers non-negative
row_CTE_trimmed_debiased_abs=np.abs(row_CTE_trimmed_debiased)
# check that this worked
print "smallest value is %s" %np.min(row_CTE_trimmed_debiased_abs)


# first overscan pixel has a fraction (1-CTE) of last imaging pixel's flux
print "simplistic CTE estimate is %s" %(1-row_CTE_trimmed_debiased_abs[1]/row_CTE_trimmed_debiased_abs[0])



In [ ]:
# try using a fit with log of abs(data). Each pixel should be I(n)=I(0)*(1-CTE)^n
logCTEarray=np.log10(row_CTE_trimmed_debiased_abs)

fig13=plt.figure(13)
plt.plot(logCTEarray,'bo-')
plt.grid()
plt.ylabel('log_10 of flux')
plt.xlabel('overscan location')

# try using more pixels:
counter=np.array([1, 2, 3])

CTEfit=np.polyfit(counter,logCTEarray[0:3],1)
CTEfit_eval=np.polyval(CTEfit,counter)

fig14=plt.figure(14)
plt.plot(counter,logCTEarray[0:3],'bo')
plt.plot(counter,CTEfit_eval,'k')
plt.xlim(xmin=0.8)
plt.xlim(xmax=3.2)
plt.grid()


# that wasn't very successful- measuring CTE is hard if signature is not evident.


In [ ]:
# construct bias-subtracted, trimmed image

# compute bias level from the overscan
print "bias value is %s" %np.mean(pixeldata5[0:2047,524:540])

pixeldata5_bias=pixeldata5-np.mean(pixeldata5[0:2047,524:540])

# keep amending array name as successive operations are applied
pixeldata5_bias_trimmed=pixeldata5_bias[0:2002,10:522]

# let's try to auto-scale the display
sortedpixels=np.sort(pixeldata5_bias_trimmed.flat)

lowestpixel=sortedpixels[0]
highestpixel=sortedpixels[-1]

print("lowest pixel value is %s") %lowestpixel
print("higheset pixel value is %s") %highestpixel

# scale to 5% and 95% values

fivepercent=sortedpixels[np.int(0.05*len(sortedpixels))]
ninetyfivepercent=sortedpixels[np.int(0.95*len(sortedpixels))]

fig16=plt.figure(16,[figwidth,figheight])
plt.imshow(pixeldata5_bias_trimmed,cmap='jet',vmin=fivepercent,vmax=ninetyfivepercent,origin='lower')
plt.colorbar()
